In [35]:
import pandas as pd
import numpy as np

#processing input training and testing dataset
def processInput( Train, Is_Test ):
    
    Train.fillna(0, inplace = True ) #remove nulls

    #Encoding
    Train["MSSubClass"] = Train["MSSubClass"].astype(object)
    OneHotColumns = list( Train.select_dtypes(include = ['object']).columns )
    OneHotDF = Train[OneHotColumns]
    OneHotDF = pd.get_dummies(OneHotDF)

    Train = Train.drop(OneHotColumns,axis=1)
    Train = Train.join(OneHotDF)
    
    if Is_Test == False :
        TrainTarget = Train['SalePrice']
        Train = Train.drop('SalePrice',axis=1)
        Train.drop("Id", axis=1)
        return Train, TrainTarget
    else :
        Id = Train['Id']
        Train.drop("Id", axis=1)
        return Train, Id

In [16]:
#remove descripancies in processed test and train dataset
def add_missing_dummy_columns( d, columns ):
    missing_cols = set( columns ) - set( d.columns )
    for c in missing_cols:
        d[c] = 0

In [17]:
Train = pd.read_csv( "train.csv" )
print Train.head()
Train, TrainTarget = processInput( Train, False )

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008     

In [29]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(Train, TrainTarget)

In [30]:
print reg.score(Train,TrainTarget)

0.933629916316


In [36]:
Test = pd.read_csv( "test.csv" )
Test, Id = processInput( Test, True );

add_missing_dummy_columns(Train, Test.columns)
add_missing_dummy_columns(Test, Train.columns)

Test = Test[Train.columns]
# TrainColumns = Train.columns
# TestColumns = Test.columns

# DiffAB = set(TrainColumns) - set(TestColumns)
# DiffAB = list(DiffAB)
# print DiffAB

# DiffBA = set(TestColumns) - set(TrainColumns)
# DiffBA = list(DiffBA)
# print DiffBA

In [48]:
predicted_values = reg.predict(Test)

In [50]:
df_final = pd.DataFrame(columns = ['Id', 'SalePrice'])
df_final['Id'] = Id
df_final['SalePrice'] = predicted_values
print df_final.head()

     Id      SalePrice
0  1461  112045.500351
1  1462  163025.284224
2  1463  188485.539722
3  1464  197947.787132
4  1465  207377.235589


In [51]:
df_final.to_csv('results.csv')